In [1]:
from flask import Flask, render_template, request,redirect, url_for
import sqlite3
import math
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
from flask import abort



In [2]:
app = Flask(__name__)
app.config['STATIC_FOLDER'] = 'static'

conn = sqlite3.connect('DOSM.db')


In [3]:
# render home.html
@app.route('/')
def home():
    return render_template('home.html')



In [4]:
from flask import Flask, render_template, request,redirect, url_for


@app.route('/result', methods=['POST', 'GET'])
def result():
    
    SQLALCHEMY_ECHO=True
    

    if request.method == 'POST':
        item_category = request.form.get('item_category')
        year = request.form.get('year')
        month = request.form.get('month')
        day = request.form.get('day')
    
    elif request.method == 'GET':
        item_category = request.args.get('item_category', default='', type=str)
        year = request.args.get('year', default='', type=str)
        month = request.args.get('month', default='', type=str)
        day = request.args.get('day', default='', type=str)
        
    
    # Split the input values if there are multiple selections
    item_category = item_category.split(",") if item_category else []
    month_list = month.split(",") if month else []
    year_list = year.split(",") if year else []
    day_list = day.split(",") if day else []
    
    print(f"item_category: {item_category}")
    print(f"month: {month}")
    print(f"year: {year}")
    print(f"day: {day}")
    print(request.method)
    
    # Validate user input
    if month and (not month.isdigit() or int(month) < 1 or int(month) > 12):
        abort(400, "Invalid value for month")
    if year and (not year.isdigit() or int(year) < 0):
        abort(400, "Invalid value for year")
    if int(year) not in [2022, 2023]:
        abort(400, "Year should be either 2022 or 2023")
    if day and (not day.isdigit() or int(day) < 1 or int(day) > 31):
        abort(400, "Invalid value for day")

    # Connect to the database
    conn = sqlite3.connect('DOSM.db')
    c = conn.cursor()

    # Set the base query
    query = "SELECT item, price, unit, item_group, item_category, premise, premise_type, state, district, address, date " \
            "FROM Price " \
            "JOIN Item ON Price.item_code = Item.item_code " \
            "JOIN Premise ON Price.premise_code = Premise.premise_code " \
            "WHERE 1 = 1 "

    # Add filters for item category, month, year, and day
    query_params = []
    if item_category:
        # Use the IN operator to filter by multiple categories
        query += f"AND item_category IN ({','.join(['?' for _ in range(len(item_category))])}) "
        query_params.extend(item_category)
    if month:
        if len(month_list) == 1:
            query += "AND month = ? "
            query_params.append(month)
        elif len(month_list) == 2:
            query += "AND month BETWEEN ? AND ? "
            query_params.extend(month_list)
    if year:
        if len(year_list) == 1:
            query += "AND year = ? "
            query_params.append(year)
        elif len(year_list) == 2:
            query += "AND year BETWEEN ? AND ? "
            query_params.extend(year_list)
    if day:
        if len(day_list) == 1:
            query += "AND day = ? "
            query_params.append(day)
        elif len(day_list) == 2:
            query += "AND day BETWEEN ? AND ? "
            query_params.extend(day_list)

    # Get the count of total records for pagination
    c.execute(f"SELECT COUNT(*) FROM ({query})", query_params)
    total_count = c.fetchone()[0]
    
   
            
    # Execute the query with the parameters
    result = []
    page = request.args.get('page',1, type=int)
    per_page = request.args.get('per_page', 50, type=int)
    
    query += "ORDER BY date DESC"
    query += f" LIMIT {per_page} OFFSET {(page-1)*per_page}"
    
    
    total_pages = math.ceil(total_count / per_page)
    
   


    c.execute(query, query_params)
    result = c.fetchall()
    # Include the input values as query parameters in the URL when the form is submitted
    


    print(f"query: {query}")
    print(f"query_params:{query_params}")
    print(f"total_count:{total_count}")
    print(f"page:{page}")
    print(f"per_page:{per_page}")
        
    # Close the database connection
    conn.close()
    
   
    # Render the result.html template with the query result and pagination information
    return render_template('result.html', result=result, page=page, per_page=per_page, total_count=total_count, total_pages=total_pages,item_category=item_category,year=year,day=day,month=month)



In [ ]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Mar/2023 14:20:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2023 14:20:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2023 14:20:30] "POST /result HTTP/1.1" 400 -
127.0.0.1 - - [24/Mar/2023 14:20:31] "GET /result HTTP/1.1" 500 -


item_category: ['AYAM']
month: 12
year: 1
day: 213321
POST
item_category: []
month: 
year: 
day: 
GET


Traceback (most recent call last):
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2551, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2531, in wsgi_app
    response = self.handle_exception(e)
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\U

item_category: ['AYAM']
month: 322
year: 2022
day: 213321
POST
item_category: []
month: 
year: 
day: 
GET


Traceback (most recent call last):
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2551, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2531, in wsgi_app
    response = self.handle_exception(e)
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\U

item_category: ['AYAM']
month: 12
year: 2022
day: 1
POST


127.0.0.1 - - [24/Mar/2023 14:20:49] "POST /result HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2023 14:20:49] "GET /result HTTP/1.1" 500 -


query: SELECT item, price, unit, item_group, item_category, premise, premise_type, state, district, address, date FROM Price JOIN Item ON Price.item_code = Item.item_code JOIN Premise ON Price.premise_code = Premise.premise_code WHERE 1 = 1 AND item_category IN (?) AND month = ? AND year = ? AND day = ? ORDER BY date DESC LIMIT 50 OFFSET 0
query_params:['AYAM', '12', '2022', '1']
total_count:29115
page:1
per_page:50
item_category: []
month: 
year: 
day: 
GET


Traceback (most recent call last):
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2551, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2531, in wsgi_app
    response = self.handle_exception(e)
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\U